In [1]:
import sys

trt_hand_dir = '/home/kiran/trt_pose_hand'
sys.path.append(trt_hand_dir)
sys.path.append('../src')

import random as Random
import trt_pose_hand
import ipywidgets
import datetime as dt
import hamsa
from jetcam.usb_camera import USBCamera
from jetcam.csi_camera import CSICamera
from jetcam.utils import bgr8_to_jpeg
from IPython.display import display
from time import sleep


from preprocessdata import preprocessdata
hand_model = trt_pose_hand.Model()
hand_model.setup(trt_hand_dir, preprocessdata)




# camera = USBCamera(width=WIDTH, height=HEIGHT, capture_fps=30, capture_device=1)
camera = CSICamera(width=hand_model.WIDTH, height=hand_model.HEIGHT, capture_fps=30)


image_w = ipywidgets.Image(format='jpeg', width=224, height=224)
display(image_w)



thing = hamsa.Hand()






Image(value=b'', format='jpeg', height='224', width='224')

In [ ]:

def get_gesture(change):
    image = change['new']
    data = hand_model.preprocess(image)
    cmap, paf = hand_model.model_trt(data)
    cmap, paf = cmap.detach().cpu(), paf.detach().cpu()
    counts, objects, peaks = hand_model.parse_objects(cmap, paf)
    joints = hand_model.preprocess_data.joints_inference(image, counts, objects, peaks)
    hand_model.draw_joints(image, joints)
    #hand_model.draw_objects(image, counts, objects, peaks)
    dist_bn_joints = hand_model.preprocess_data.find_distance(joints)
    gesture = hand_model.clf.predict([dist_bn_joints,[0]*hand_model.num_parts*hand_model.num_parts])
    gesture_joints = gesture[0]
    hand_model.preprocess_data.prev_queue.append(gesture_joints)
    hand_model.preprocess_data.prev_queue.pop(0)
    hand_model.preprocess_data.print_label(image, hand_model.preprocess_data.prev_queue, hand_model.gesture_type)
    image_w.value = bgr8_to_jpeg(image)
    with open('jetcam.jpeg', 'wb') as f:
        f.write(image_w.value)
    return hand_model.gesture_type[gesture_joints-1]









def clear_thumb():
    if thing.curl_thumb_pos() < 0.19:
        thing.curl_thumb(.19,100)
        sleep(.1)


def idle_wiggle():
    thing.wiggle_middle(0.6,100)
    thing.wiggle_pinky(0.5,100)
    thing.wiggle_ring(.45,100)
    thing.wiggle_index(.5,100)


def flick_up(fingers, time):
    thing.curl_wrist(0.65, time)
    for fin in fingers:
        if fin is 'pinky':
            thing.curl_pinky(1,time)
        if fin is 'ring':
            thing.curl_ring(1,time)
        if fin is 'middle':
            thing.curl_middle(1,time)
        if fin is 'index':
            thing.curl_index(1,time)
        sleep(.1)


def scissors(time):
    clear_thumb()
    idle_wiggle()
    delay = .1
    flick_up(['middle', 'index'], time)
    sleep(delay)
    thing.wiggle_middle(.7,100)
    thing.wiggle_index(.3,100)
    thing.curl_pinky(0,time)
    sleep(delay)
    thing.curl_ring(0,time)
    sleep(delay)
    thing.wiggle_thumb(.5,time)
    thing.curl_thumb(0,time)
    thing.curl_wrist(.6, 400)

def rock(time):
    clear_thumb()
    idle_wiggle()
    delay = .2
    thing.curl_pinky(0, time)
    sleep(delay)
    thing.curl_ring(0, time)
    sleep(delay)
    thing.curl_middle(0, time)
    sleep(delay)
    thing.curl_index(0, time)
    sleep(delay)
    thing.wiggle_thumb(0,time)
    sleep(.1)
    thing.curl_thumb(0.2,time)  
    sleep(time/1000)

def paper(time):
    idle_wiggle()
    clear_thumb()
    flick_up(['pinky','ring','middle','index'], time)
    thing.wiggle_thumb(0,100)
    thing.curl_thumb(1,time)
    sleep(.3)
    thing.curl_wrist(.6,100)


def ok(time):
    clear_thumb()
    idle_wiggle()
    delay = .1
    flick_up(['pinky','ring','middle'], 500)
    sleep(delay)
    thing.curl_index(.25,time)
    sleep(delay)
    thing.wiggle_thumb(.7,time)
    thing.curl_thumb(.13,time)
    thing.curl_wrist(.6, 400)
    
    
def one(time):
    idle_wiggle()
    clear_thumb()
    flick_up(['pinky'], time)

def two(time):
    idle_wiggle()
    clear_thumb()
    flick_up(['pinky', 'ring'], time)

def three(time):
    idle_wiggle()
    clear_thumb()
    flick_up(['pinky','ring','middle'], time)

def one_two_three(time):
    rock(time)
    delay = 1
    sleep(delay)
    one(time)
    sleep(delay)
    two(time)
    sleep(delay)
    three(delay)
    sleep(delay)


def win(time):
    ok(time)

def lost(time):
    paper(time)
    thing.curl_wrist(0.05, time)


def draw(time):
    rock(time)
    three(time)



choice: int = Random.randint(1, 3)
one_two_three(1000)
if choice == 1:
    rock(1000)
elif choice == 2:
    paper(1000)
elif choice == 3:
    scissors(1000)





camera.running = True
print(f'Possible gestures: {hand_model.gesture_type}')
this_gesture = ''
current_gesture = ''
last_change = dt.datetime.now()
read = True
while read:
    last_gesture = this_gesture
    this_gesture = get_gesture({'new': camera.value})
    if this_gesture is not last_gesture:
        last_change = dt.datetime.now()
    long_enough = (dt.datetime.now()-last_change).total_seconds() > .7
    if long_enough and (this_gesture is not current_gesture):
        print(f'Current gesture: {this_gesture}'.ljust(30), end='\r')
        if this_gesture == 'fist' and choice == 1:
            draw(1000)
        elif this_gesture == 'fist' and choice == 2:
            win(1000)
        elif this_gesture == 'fist' and choice == 3:
            lost(1000)
        elif this_gesture == 'stop' and choice == 1:
            lost(1000)
        elif this_gesture == 'stop' and choice == 2:
            draw(1000)
        elif this_gesture == 'stop' and choice == 3:
            win(1000)
        elif this_gesture == 'peace' and choice == 1:
            win(1000)
        elif this_gesture == 'peace' and choice == 2:
            lost(1000)
        elif this_gesture == 'peace' and choice == 3:
            draw(1000)

        if this_gesture == 'fist' or this_gesture == 'stop' or this_gesture == 'peace':
            read = False








camera.running = False

Possible gestures: ['fist', 'pan', 'stop', 'peace', 'ok', 'no hand', 'no hand']


Exception in thread Thread-4:
Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.6/dist-packages/jetcam-0.0.0-py3.6.egg/jetcam/camera.py", line 34, in _capture_frames
    self.value = self._read()
  File "/usr/local/lib/python3.6/dist-packages/jetcam-0.0.0-py3.6.egg/jetcam/csi_camera.py", line 40, in _read
    raise RuntimeError('Could not read image from camera')
RuntimeError: Could not read image from camera

